In [ ]:
%matplotlib inline 
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report


#K-Means import
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from time import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def bench_k_means(estimator, name, data,labels=None):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=None)))
    return estimator

# K-Means Fold 1

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
traindata=pd.read_csv("../data/tsv/test1.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("../data/tsv/train1.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)
clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('tfidf', TfidfTransformer(use_idf=True))])
clf.fit(X_train)


X_train=(clf.transform(X_train)).toarray()
X_test=clf.transform(X_test)

data=X_train
labels=np.array(Y_train)-1
# print(labels)

# Do K-Means
for rstate in [16]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))
    cfmatrix=confusion_matrix(np.array(Y_test)-1,ypred)
    print(cfmatrix)
    

#     print("Random Initialization")
#     ypred=km1_random.predict(X_test)
#     print(classification_report(np.array(Y_test)-1,ypred,digits=5))


#     print("PCA Components Initialization")
#     ypred=km1_pca.predict(X_test)
#     print(classification_report(np.array(Y_test)-1,ypred,digits=5))

In [ ]:
# Confusion Matrix Plot

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt


df_cm = pd.DataFrame(cfmatrix, index=["stage 1", "stage 2", "stage 3", "stagte 4"], columns=["stage 1", "stage 2", "stage 3", "stagte 4"])

from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.collections import QuadMesh
import seaborn as sn


def get_new_fig(fn, figsize=[9,9]):
    """ Init graphics """
    fig1 = plt.figure(fn, figsize)
    ax1 = fig1.gca()   #Get Current Axis
    ax1.cla() # clear existing plot
    return fig1, ax1
#

def configcell_text_and_colors(array_df, lin, col, oText, facecolors, posi, fz, fmt, show_null_values=0):
    """
      config cell text and colors
      and return text elements to add and to dell
      @TODO: use fmt
    """
    text_add = []; text_del = [];
    cell_val = array_df[lin][col]
    tot_all = array_df[-1][-1]
    per = (float(cell_val) / tot_all) * 100
    curr_column = array_df[:,col]
    ccl = len(curr_column)

    #last line  and/or last column
    if(col == (ccl - 1)) or (lin == (ccl - 1)):
        #tots and percents
        if(cell_val != 0):
            if(col == ccl - 1) and (lin == ccl - 1):
                tot_rig = 0
                for i in range(array_df.shape[0] - 1):
                    tot_rig += array_df[i][i]
                per_ok = (float(tot_rig) / cell_val) * 100
            elif(col == ccl - 1):
                tot_rig = array_df[lin][lin]
                per_ok = (float(tot_rig) / cell_val) * 100
            elif(lin == ccl - 1):
                tot_rig = array_df[col][col]
                per_ok = (float(tot_rig) / cell_val) * 100
            per_err = 100 - per_ok
        else:
            per_ok = per_err = 0

        per_ok_s = ['%.2f%%'%(per_ok), '100%'] [per_ok == 100]

        #text to DEL
        text_del.append(oText)

        #text to ADD
        font_prop = fm.FontProperties(weight='bold', size=fz)
        text_kwargs = dict(color='w', ha="center", va="center", gid='sum', fontproperties=font_prop)
        lis_txt = ['%d'%(cell_val), per_ok_s, '%.2f%%'%(per_err)]
        lis_kwa = [text_kwargs]
        dic = text_kwargs.copy(); dic['color'] = 'g'; lis_kwa.append(dic);
        dic = text_kwargs.copy(); dic['color'] = 'r'; lis_kwa.append(dic);
        lis_pos = [(oText._x, oText._y-0.3), (oText._x, oText._y), (oText._x, oText._y+0.3)]
        for i in range(len(lis_txt)):
            newText = dict(x=lis_pos[i][0], y=lis_pos[i][1], text=lis_txt[i], kw=lis_kwa[i])
            #print 'lin: %s, col: %s, newText: %s' %(lin, col, newText)
            text_add.append(newText)
        #print '\n'

        #set background color for sum cells (last line and last column)
        carr = [0.27, 0.30, 0.27, 1.0]
        if(col == ccl - 1) and (lin == ccl - 1):
            carr = [0.17, 0.20, 0.17, 1.0]
        facecolors[posi] = carr

    else:
        if(per > 0):
            txt = '%s\n%.2f%%' %(cell_val, per)
        else:
            if(show_null_values == 0):
                txt = ''
            elif(show_null_values == 1):
                txt = '0'
            else:
                txt = '0\n0.0%'
        oText.set_text(txt)

        #main diagonal
        if(col == lin):
            #set color of the textin the diagonal to white
            oText.set_color('w')
            # set background color in the diagonal to blue
            facecolors[posi] = [0.35, 0.8, 0.55, 1.0]
        else:
            oText.set_color('r')

    return text_add, text_del
#

def insert_totals(df_cm):
    """ insert total column and line (the last ones) """
    sum_col = []
    for c in df_cm.columns:
        sum_col.append( df_cm[c].sum() )
    sum_lin = []
    for item_line in df_cm.iterrows():
        sum_lin.append( item_line[1].sum() )
    df_cm['sum_lin'] = sum_lin
    sum_col.append(np.sum(sum_lin))
    df_cm.loc['sum_col'] = sum_col
    #print ('\ndf_cm:\n', df_cm, '\n\b\n')
#

def pretty_plot_confusion_matrix(df_cm, annot=True, cmap="Oranges", fmt='.2f', fz=11,
      lw=0.5, cbar=False, figsize=[8,8], show_null_values=0, pred_val_axis='y'):
    """
      print conf matrix with default layout (like matlab)
      params:
        df_cm          dataframe (pandas) without totals
        annot          print text in each cell
        cmap           Oranges,Oranges_r,YlGnBu,Blues,RdBu, ... see:
        fz             fontsize
        lw             linewidth
        pred_val_axis  where to show the prediction values (x or y axis)
                        'col' or 'x': show predicted values in columns (x axis) instead lines
                        'lin' or 'y': show predicted values in lines   (y axis)
    """
    if(pred_val_axis in ('col', 'x')):
        xlbl = 'Predicted'
        ylbl = 'Actual'
    else:
        xlbl = 'Actual'
        ylbl = 'Predicted'
        df_cm = df_cm.T

    # create "Total" column
    insert_totals(df_cm)

    #this is for print allways in the same window
    fig, ax1 = get_new_fig('Conf matrix default', figsize)

    #thanks for seaborn
    ax = sn.heatmap(df_cm, annot=annot, annot_kws={"size": fz}, linewidths=lw, ax=ax1,
                    cbar=cbar, cmap=cmap, linecolor='w', fmt=fmt)

    #set ticklabels rotation
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, fontsize = 10)
    ax.set_yticklabels(ax.get_yticklabels(), rotation = 25, fontsize = 10)

    # Turn off all the ticks
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    #face colors list
    quadmesh = ax.findobj(QuadMesh)[0]
    facecolors = quadmesh.get_facecolors()

    #iter in text elements
    array_df = np.array( df_cm.to_records(index=False).tolist() )
    text_add = []; text_del = [];
    posi = -1 #from left to right, bottom to top.
    for t in ax.collections[0].axes.texts: #ax.texts:
        pos = np.array( t.get_position()) - [0.5,0.5]
        lin = int(pos[1]); col = int(pos[0]);
        posi += 1
        #print ('>>> pos: %s, posi: %s, val: %s, txt: %s' %(pos, posi, array_df[lin][col], t.get_text()))

        #set text
        txt_res = configcell_text_and_colors(array_df, lin, col, t, facecolors, posi, fz, fmt, show_null_values)

        text_add.extend(txt_res[0])
        text_del.extend(txt_res[1])

    #remove the old ones
    for item in text_del:
        item.remove()
    #append the new ones
    for item in text_add:
        ax.text(item['x'], item['y'], item['text'], **item['kw'])

    #titles and legends
    ax.set_title('Confusion matrix')
    ax.set_xlabel(xlbl)
    ax.set_ylabel(ylbl)
    plt.tight_layout()  #set layout slim
    plt.show()
#

def plot_confusion_matrix_from_data(y_test, predictions, columns=None, annot=True, cmap="Oranges",
      fmt='.2f', fz=11, lw=0.5, cbar=False, figsize=[8,8], show_null_values=0, pred_val_axis='lin'):
    """
        plot confusion matrix function with y_test (actual values) and predictions (predic),
        whitout a confusion matrix yet
    """
    from sklearn.metrics import confusion_matrix
    from pandas import DataFrame

    #data
    if(not columns):
        #labels axis integer:
        ##columns = range(1, len(np.unique(y_test))+1)
        #labels axis string:
        from string import ascii_uppercase
        columns = ['class %s' %(i) for i in list(ascii_uppercase)[0:len(np.unique(y_test))]]

    confm = confusion_matrix(y_test, predictions)
    cmap = 'Oranges';
    fz = 11;
    figsize=[9,9];
    show_null_values = 2
    df_cm = DataFrame(confm, index=columns, columns=columns)
    pretty_plot_confusion_matrix(df_cm, fz=fz, cmap=cmap, figsize=figsize, show_null_values=show_null_values, pred_val_axis=pred_val_axis)
#

plot_confusion_matrix_from_data(np.array(Y_test)-1,ypred,columns=["Overview","Congitive Connection","Relation to KBAI Class","Agent Reasoning"])

# K-MeansFold 2

In [ ]:
from sklearn.dummy import DummyClassifier
traindata=pd.read_csv("../data/tsv/test2.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("../data/tsv/train2.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)
clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('tfidf', TfidfTransformer(use_idf=True))])
clf.fit(X_train)


X_train=(clf.transform(X_train)).toarray()
X_test=clf.transform(X_test)

data=X_train
labels=np.array(Y_train)-1
# print(labels)

# Do K-Means
# Do K-Means
for rstate in [2]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))


# Testing Autoclustering with Elbow Method


In [ ]:
# FOld -1

from sklearn.dummy import DummyClassifier
traindata=pd.read_csv("../data/tsv/test1.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("../data/tsv/train1.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)
clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('tfidf', TfidfTransformer(use_idf=True))])
clf.fit(X_train)


X_train=(clf.transform(X_train)).toarray()
X_test=clf.transform(X_test)

data=X_train
labels=np.array(Y_train)-1
# print(labels)


distortions = []
for i in range(1, 100):
    km = KMeans(init='k-means++', n_clusters=i, n_init=10,random_state=rstate)
    km.fit(data)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 100), distortions, marker='-')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

# K-Means with language model representations from RQ 2.2

In [ ]:
import pickle

#change modelfolder path to get results across different embeddings

modelfolder="../../RQ2.2/data/bert/fold1/"

X_train= pickle.load(open(modelfolder+'xtrain.pkl', 'rb'))
Y_train= pickle.load(open(modelfolder+'ytrain.pkl', 'rb'))
X_test= pickle.load(open(modelfolder+'xtest.pkl', 'rb'))
Y_test= pickle.load(open(modelfolder+'ytest.pkl', 'rb'))


data=X_train
labels=np.array(Y_train)
# print(labels)

# Do K-Means
for rstate in [23]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

In [ ]:
import pickle

modelfolder="../../RQ2.2/data/bert/fold1/"

X_train= pickle.load(open(modelfolder+'xtrain.pkl', 'rb'))
Y_train= pickle.load(open(modelfolder+'ytrain.pkl', 'rb'))
X_test= pickle.load(open(modelfolder+'xtest.pkl', 'rb'))
Y_test= pickle.load(open(modelfolder+'ytest.pkl', 'rb'))



data=X_train
labels=np.array(Y_train)
# print(labels)

# Do K-Means
for rstate in [1]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))